In [1]:

include("node.jl")
include("edges.jl")
include("graph.jl")
include("read_stsp.jl")
include("comp_connexes.jl")
include("main.jl")
include("queue.jl")

last_in_first_out_priority_queue

In [2]:
tree_root = Tree("1", 0)
child1 = Tree("2", 1)
child2 = Tree("3", 2)
child3 = Tree("4", 3)
child4 = Tree("5", 4)
first_row = [ child1, child2, child3]
second_row = [child4]
for child in first_row
    change_parent!(child, tree_root)
end
for child in second_row
    change_parent!(child, child3)
end

In [3]:
show(tree_root)

Node 1 has  rank 0
listing of children : 
node_visited:2 its parent is 1 its rank is 0
node_visited:3 its parent is 1 its rank is 0
node_visited:4 its parent is 1 its rank is 0
node_visited:5 its parent is 4 its rank is 0


In [4]:


"""Implementation du premier heuristic"""
function rank_union!(tree_1::Tree{T}, tree_2::Tree{T}) where T
    if rank(tree_1) == rank(tree_2)
      change_parent!(tree_2, tree_1)
      change_rank!(tree_1, rank(tree_1) + 1)
      return tree_1
    elseif rank(tree_1) > rank(tree_2)
      change_parent!(tree_2, tree_21)
      return tree_1
    else
      change_parent!(tree_1, tree_2)
      return tree_2
    end
end

rank_union!

In [5]:
tree_A = Tree("A", 0)
tree_B = Tree("B", 1)
new_tree = rank_union!(tree_A, tree_B)
tree_C = Tree("C", 2)
tree_D = Tree("D", 3)
second_tree = rank_union!(tree_C, tree_D)
final_tree = rank_union!(new_tree, second_tree)
show(final_tree)


Node A has  rank 2
listing of children : 
node_visited:B its parent is A its rank is 0


node_visited:C its parent is A its rank is 1
node_visited:D its parent is C its rank is 0


In [6]:
function find_root(tree::Tree{T}) where T
    #root is the only node without a parent
    if isnothing(parent(tree))
        return tree
    else
        return find_root(parent(tree))
    end
end


"""Implementation du deuxieme heuristic"""
function path_compression(tree::Tree{T}) where T
    #la racine est le seul noeud sans parent
    if isnothing(parent(tree))
        return tree
    else
        root =  path_compression(parent(tree))
        change_parent!(tree, root)
        return root
    end
end

path_compression

In [7]:
graph =  Graph("graphe1",Node{Float64}[],Edge{Float64,Float64}[])
node_A = Node("A", 0.0)
node_B = Node("B", 0.0)
node_C = Node("C", 0.0)
node_D = Node("D", 0.0)
node_E = Node("E", 0.0)
node_F = Node("F", 0.0)
node_G = Node("G", 0.0)
node_H = Node("H", 0.0)
node_I = Node("I", 0.0)
graph_nodes = Vector{Node}([node_A, node_B, node_C, node_D, node_E, node_F, node_G, node_H, node_I]);
for node in graph_nodes
    add_node!(graph, node)
end
edge_AB = Edge(node_A, node_B, 4.0)
edge_BC = Edge(node_B, node_C, 8.0)
edge_CD = Edge(node_C, node_D, 7.0)
edge_DE = Edge(node_D, node_E, 9.0)
edge_EF = Edge(node_E, node_F, 10.0)
edge_DF = Edge(node_D, node_F, 14.0)
edge_CF = Edge(node_C, node_F, 4.0)
edge_CI = Edge(node_C, node_I, 2.0)
edge_GF = Edge(node_G, node_F, 2.0)
edge_GI = Edge(node_G, node_I, 6.0)
edge_HG = Edge(node_H, node_G, 1.0)
edge_HI = Edge(node_H, node_I, 6.0)
edge_HA = Edge(node_H, node_A, 8.0)
edge_HB = Edge(node_H, node_B, 11.0)
graph_edges = Vector{Edge}([edge_AB, edge_BC, edge_CD, edge_DE, edge_EF, edge_DF, edge_CF, edge_CI, edge_GF, edge_GI, edge_HG, edge_HI, edge_HA, edge_HB])
for edge in graph_edges
    add_edge!(graph, edge)
end
show(graph)

Graph graphe1 has 9 nodes.
Node A, data: 0.0
Node B, data: 0.0
Node C, data: 0.0
Node D, data: 0.0
Node E, data: 0.0
Node F, data: 0.0
Node G, data: 0.0
Node H, data: 0.0
Node I, data: 0.0
Graph graphe1 has 14 edges.
Arête de A à B, poids: 4.0
Arête de B à C, poids: 8.0
Arête de C à D, poids: 7.0
Arête de D à E, poids: 9.0
Arête de E à F, poids: 10.0
Arête de D à F, poids: 14.0
Arête de C à F, poids: 4.0
Arête de C à I, poids: 2.0
Arête de G à F, poids: 2.0
Arête de G à I, poids: 6.0
Arête de H à G, poids: 1.0
Arête de H à I, poids: 6.0
Arête de H à A, poids: 8.0
Arête de H à B, poids: 11.0


In [25]:
function neighbors(node::Node, graph::Graph)
    neighbors = []
    for edge in edges(graph)
        if edge.node1 == node
            push!(neighbors, edge.node2)
        elseif edge.node2 == node
            push!(neighbors, edge.node1)
        end
    end
    return unique!(neighbors)
end

function neighbors_and_distance(node::Node, graph::Graph)
    neighbor_data = Dict()
    for edge in edges(graph)
        if edge.node1 == node
            neighbor_data[edge.node2] = edge.weight
        elseif edge.node2 == node
            neighbor_data[edge.node1] = edge.weight
        end
        
    end
    return neighbor_data
end

function incident_edges(node::Node, graph::Graph)
    incident_edges = []
    for edge in edges(graph)
        if edge.node1 == node || edge.node2 == node
            push!(incident_edges, edge)
        end
    end
    return incident_edges
end

function prims_algorithm(graph::Graph, start_node_name::String = nothing) 
    #initialisation
    if isnothing(start_node_name)
        start_node_name = name(nodes(graph)[1])
    end
    #initialisation de la file de priorite
    priority_queue = PriorityQueue{PriorityItem{Tree{Node{Float64}}}}()
    for node in nodes(graph)
        if name(node) == start_node_name
                node.data = 0.0
        else
            node.data = Inf
        end
        blank_tree = Tree(name(node), node)
        priority_item = PriorityItem( data(node), blank_tree)
        push!(priority_queue, priority_item)
    end
    #initialisation de la racine de l'arbre
    root = poplast!(priority_queue)
    tree = root.data
    #boucle principale
    while !is_empty(priority_queue)
        node = data(tree)
        for  (neighbor, edge_weight) in neighbors_and_distance(node, graph)
            println("neighbor: ", name(neighbor), " edge_weight: ", edge_weight)
            for priority_item in priority_queue.items
                if name(priority_item.data) == name(neighbor)
                    if  edge_weight < neighbor.data
                        neighbor.data = edge_weight
                        change_parent!(priority_item.data, tree)
                        change_rank!(priority_item.data, edge_weight)
                        priority!(priority_item, edge_weight)
                        println("priority_item: ", priority_item.data.name, " ", priority_item.priority)
                    end
                end
            end
        end
        u = poplast!(priority_queue)
        tree= u.data
        println("tree: ", tree)        
    end
    return root.data
end

prims_algorithm (generic function with 2 methods)

In [26]:
my_tree = prims_algorithm(graph, "A")


neighbor: H edge_weight: 8.0
priority_item: H 8.0
neighbor: B edge_weight: 4.0
priority_item: B 4.0
tree: Tree{Node{Float64}}("B", Node{Float64}("B", 4.0), Tree{Node{Float64}}("A", Node{Float64}("A", 0.0), nothing, Tree{Node{Float64}}[Tree{Node{Float64}}("H", Node{Float64}("H", 8.0), Tree{Node{Float64}}(#= circular reference @-3 =#), Tree{Node{Float64}}[], 8.0), Tree{Node{Float64}}(#= circular reference @-3 =#)], 0), Tree{Node{Float64}}[], 4.0)
neighbor: C edge_weight: 8.0
priority_item: C 8.0
neighbor: H edge_weight: 11.0
neighbor: A edge_weight: 4.0
tree: Tree{Node{Float64}}("C", Node{Float64}("C", 8.0), Tree{Node{Float64}}("B", Node{Float64}("B", 4.0), Tree{Node{Float64}}("A", Node{Float64}("A", 0.0), nothing, Tree{Node{Float64}}[Tree{Node{Float64}}("H", Node{Float64}("H", 8.0), Tree{Node{Float64}}(#= circular reference @-3 =#), Tree{Node{Float64}}[], 8.0), Tree{Node{Float64}}(#= circular reference @-3 =#)], 0), Tree{Node{Float64}}[Tree{Node{Float64}}(#= circular reference @-3 =#)],

Tree{Node{Float64}}("A", Node{Float64}("A", 0.0), nothing, Tree{Node{Float64}}[Tree{Node{Float64}}("B", Node{Float64}("B", 4.0), Tree{Node{Float64}}(#= circular reference @-3 =#), Tree{Node{Float64}}[Tree{Node{Float64}}("C", Node{Float64}("C", 8.0), Tree{Node{Float64}}(#= circular reference @-3 =#), Tree{Node{Float64}}[Tree{Node{Float64}}("D", Node{Float64}("D", 7.0), Tree{Node{Float64}}(#= circular reference @-3 =#), Tree{Node{Float64}}[Tree{Node{Float64}}("E", Node{Float64}("E", 9.0), Tree{Node{Float64}}(#= circular reference @-3 =#), Tree{Node{Float64}}[], 9.0)], 7.0), Tree{Node{Float64}}("F", Node{Float64}("F", 4.0), Tree{Node{Float64}}(#= circular reference @-3 =#), Tree{Node{Float64}}[Tree{Node{Float64}}("G", Node{Float64}("G", 2.0), Tree{Node{Float64}}(#= circular reference @-3 =#), Tree{Node{Float64}}[Tree{Node{Float64}}("H", Node{Float64}("H", 1.0), Tree{Node{Float64}}(#= circular reference @-3 =#), Tree{Node{Float64}}[], 1.0)], 2.0)], 4.0), Tree{Node{Float64}}("I", Node{Float

In [20]:
show(my_tree)

Node A has  rank 0
listing of children : 
node_visited:B its parent is A its rank is 4.0
node_visited:C its parent is B its rank is 8.0
node_visited:F its parent is C its rank is 4.0
node_visited:I its parent is C its rank is 2.0
node_visited:G its parent is F its rank is 2.0
node_visited:E its parent is F its rank is 10.0
node_visited:H its parent is G its rank is 1.0
node_visited:D its parent is E its rank is 9.0


In [27]:
show(my_tree)

Node A has  rank 0
listing of children : 
node_visited:B its parent is A its rank is 4.0
node_visited:C its parent is B its rank is 8.0
node_visited:D its parent is C its rank is 7.0
node_visited:F its parent is C its rank is 4.0
node_visited:I its parent is C its rank is 2.0
node_visited:E its parent is D its rank is 9.0
node_visited:G its parent is F its rank is 2.0
node_visited:H its parent is G its rank is 1.0


In [11]:
q = PriorityQueue{PriorityItem{Int}}()
push!(q, PriorityItem(2, 3));
push!(q, PriorityItem(4, 3));
push!(q, PriorityItem(1, 3));

for i in q.items
    println(i)
end

PriorityItem{Int64}(2, 3)
PriorityItem{Int64}(4, 3)
PriorityItem{Int64}(1, 3)


In [12]:
neighbors_and_distance(node_A, graph)

Dict{Any, Any} with 2 entries:
  Node{Float64}("H", Inf) => 8.0
  Node{Float64}("B", Inf) => 4.0

In [13]:
# function graph_from_incident_matrix(fp)
#     #fp is the file path
#     #we read the file
#     file = open(fp)
#     lines = readlines(file)
#     close(file)
#     #we create the graph
#     graph =  Graph("graphe1",Node{Int}[],Edge{Int,Int}[])
#     #we add the nodes
#     for i in 2:length(lines)
#         node_name = string(lines[i][1])
#         add_node!(graph, Node(node_name, 0))
#     end
#     #we add the edges
#     re = r"^[+-]?([0-9]+([.][0-9]*)?|[.][0-9]+)$"

#     for i in 2:length(lines)
#         println("i ", i)
#         println("lines1 ", lines[i])
#         println("lines1[1] ", lines[i][3])
#         for k in lines[i]
#             println("k ", k)
#         end
#         for j in i+1:length(lines)-1
#             println("j", j)
#             println("lines2", lines[i][j])
#             if occursin(re, string(lines[i][j]) )
#                 edge_weight = parse(Int64, lines[i][j])
#                 if edge_weight > 0
#                     edge = Edge(nodes(graph)[i], nodes(graph)[j], edge_weight)
#                     add_edge!(graph, edge)
#                 end
#             end
#         end
#     end
#     return graph
# end

In [14]:
# demo_graph = graph_from_incident_matrix("practice_graph.csv")
# show(demo_graph)

In [15]:
my_graph = graph_from_tsp( "../../instances/stsp/bayg29.tsp", "test")

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓


┌ Warning: ("Le noeud", "1", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:35
┌ Warning: ("Le noeud", "2", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:39
┌ Warning: ("Le noeud", "3", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:39
┌ Warning: ("Le noeud", "4", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:39
┌ Warning: ("Le noeud", "5", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:39
┌ Warning: ("Le noeud", "6", " n'était pas da

Graph{Int64, Int64}("test", Node{Int64}[Node{Int64}("1", 0), Node{Int64}("2", 0), Node{Int64}("3", 0), Node{Int64}("4", 0), Node{Int64}("5", 0), Node{Int64}("6", 0), Node{Int64}("7", 0), Node{Int64}("8", 0), Node{Int64}("9", 0), Node{Int64}("10", 0)  …  Node{Int64}("20", 0), Node{Int64}("21", 0), Node{Int64}("22", 0), Node{Int64}("23", 0), Node{Int64}("24", 0), Node{Int64}("25", 0), Node{Int64}("26", 0), Node{Int64}("27", 0), Node{Int64}("28", 0), Node{Int64}("29", 0)], Edge{Int64, Int64}[Edge{Int64, Int64}(Node{Int64}("1", 0), Node{Int64}("2", 0), 97), Edge{Int64, Int64}(Node{Int64}("1", 0), Node{Int64}("3", 0), 205), Edge{Int64, Int64}(Node{Int64}("1", 0), Node{Int64}("4", 0), 139), Edge{Int64, Int64}(Node{Int64}("1", 0), Node{Int64}("5", 0), 86), Edge{Int64, Int64}(Node{Int64}("1", 0), Node{Int64}("6", 0), 60), Edge{Int64, Int64}(Node{Int64}("1", 0), Node{Int64}("7", 0), 220), Edge{Int64, Int64}(Node{Int64}("1", 0), Node{Int64}("8", 0), 65), Edge{Int64, Int64}(Node{Int64}("1", 0), N

In [16]:
show(my_graph)

Graph test has 29 nodes.
Node 1, data: 0
Node 2, data: 0
Node 3, data: 0
Node 4, data: 0
Node 5, data: 0
Node 6, data: 0
Node 7, data: 0
Node 8, data: 0
Node 9, data: 0
Node 10, data: 0
Node 11, data: 0
Node 12, data: 0
Node 13, data: 0
Node 14, data: 0
Node 15, data: 0
Node 16, data: 0
Node 17, data: 0
Node 18, data: 0
Node 19, data: 0
Node 20, data: 0
Node 21, data: 0
Node 22, data: 0
Node 23, data: 0
Node 24, data: 0
Node 25, data: 0
Node 26, data: 0
Node 27, data: 0
Node 28, data: 0
Node 29, data: 0
Graph test has 406 edges.
Arête de 1 à 2, poids: 97
Arête de 1 à 3, poids: 205
Arête de 1 à 4, poids: 139
Arête de 1 à 5, poids: 86
Arête de 1 à 6, poids: 60
Arête de 1 à 7, poids: 220
Arête de 1 à 8, poids: 65
Arête de 1 à 9, poids: 111
Arête de 1 à 10, poids: 115
Arête de 1 à 11, poids: 227
Arête de 1 à 12, poids: 95
Arête de 1 à 13, poids: 82
Arête de 1 à 14, poids: 225
Arête de 1 à 15, poids: 168
Arête de 1 à 16, poids: 103
Arête de 1 à 17, poids: 266
Arête de 1 à 18, poids: 205
Arê


Arête de 16 à 26, poids: 218
Arête de 16 à 27, poids: 48
Arête de 16 à 28, poids: 127
Arête de 16 à 29, poids: 214
Arête de 17 à 18, poids: 61
Arête de 17 à 19, poids: 155
Arête de 17 à 20, poids: 157
Arête de 17 à 21, poids: 235
Arête de 17 à 22, poids: 47
Arête de 17 à 23, poids: 305
Arête de 17 à 24, poids: 243
Arête de 17 à 25, poids: 186
Arête de 17 à 26, poids: 282
Arête de 17 à 27, poids: 261
Arête de 17 à 28, poids: 300
Arête de 17 à 29, poids: 252
Arête de 18 à 19, poids: 105
Arête de 18 à 20, poids: 100
Arête de 18 à 21, poids: 176
Arête de 18 à 22, poids: 66
Arête de 18 à 23, poids: 253
Arête de 18 à 24, poids: 183
Arête de 18 à 25, poids: 146
Arête de 18 à 26, poids: 231
Arête de 18 à 27, poids: 203
Arête de 18 à 28, poids: 239
Arête de 18 à 29, poids: 204
Arête de 19 à 20, poids: 113
Arête de 19 à 21, poids: 152
Arête de 19 à 22, poids: 127
Arête de 19 à 23, poids: 150
Arête de 19 à 24, poids: 106
Arête de 19 à 25, poids: 52
Arête de 19 à 26, poids: 235
Arête de 19 à 27, 